# Installing Java 8

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!java -version

!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!update-alternatives --set javac /usr/lib/jvm/java-8-openjdk-amd64/bin/javac
!update-alternatives --set jps /usr/lib/jvm/java-8-openjdk-amd64/bin/jps
!java -version

#Finding the default Java path
!readlink -f /usr/bin/java | sed "s:bin/java::"
!apt-get install openssh-server -qq > /dev/null
!service ssh start

!grep Port /etc/ssh/sshd_config

#Creating a new rsa key pair with empty password
!ssh-keygen -t rsa -P "" -f ~/.ssh/id_rsa <<< y

# See id_rsa.pub content
!more /root/.ssh/id_rsa.pub

#Copying the key to autorized keys
!cat $HOME/.ssh/id_rsa.pub > $HOME/.ssh/authorized_keys
#Changing the permissions on the key
!chmod 0600 ~/.ssh/authorized_keys

#Conneting with the local machine
!ssh -o StrictHostKeyChecking=no localhost uptime

# https://archive.apache.org/dist/hadoop/common/hadoop-3.2.3/hadoop-3.2.3.tar.gz
# mirror download: http://mirrors.cloud.aliyuncs.com/apache/hadoop/common/hadoop-3.2.4/hadoop-3.2.4.tar.gz

#Downloading Hadoop 3.2.4
!wget -q https://downloads.apache.org/hadoop/common/hadoop-3.2.4/hadoop-3.2.4.tar.gz

#Untarring the file
!sudo tar -xzf hadoop-3.2.4.tar.gz
#Removing the tar file
!rm hadoop-3.2.4.tar.gz


#Copying the hadoop files to user/local
!cp -r hadoop-3.2.4/ /usr/local/
#-r copy directories recursively

#Adding JAVA_HOME directory to hadoop-env.sh file
!sed -i '/export JAVA_HOME=/a export JAVA_HOME=\/usr\/lib\/jvm\/java-8-openjdk-amd64' /usr/local/hadoop-3.2.4/etc/hadoop/hadoop-env.sh

import os
#Creating environment variables
#Creating Hadoop home variable

os.environ["HADOOP_HOME"] = "/usr/local/hadoop-3.2.4"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["JRE_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre"
os.environ["PATH"] += f'{os.environ["JAVA_HOME"]}/bin:{os.environ["JRE_HOME"]}/bin:{os.environ["HADOOP_HOME"]}/sbin'

#Dowloading text example to use as input
!wget -q https://www.mirrorservice.org/sites/ftp.ibiblio.org/pub/docs/books/gutenberg/1/0/101/101.txt

openjdk version "1.8.0_442"
OpenJDK Runtime Environment (build 1.8.0_442-8u442-b06~us1-0ubuntu1~22.04-b06)
OpenJDK 64-Bit Server VM (build 25.442-b06, mixed mode)
openjdk version "1.8.0_442"
OpenJDK Runtime Environment (build 1.8.0_442-8u442-b06~us1-0ubuntu1~22.04-b06)
OpenJDK 64-Bit Server VM (build 25.442-b06, mixed mode)
/usr/lib/jvm/java-8-openjdk-amd64/jre/
 * Starting OpenBSD Secure Shell server sshd
   ...done.
#Port 22
#GatewayPorts no
Generating public/private rsa key pair.
/root/.ssh/id_rsa already exists.
Overwrite (y/n)? Your identification has been saved in /root/.ssh/id_rsa
Your public key has been saved in /root/.ssh/id_rsa.pub
The key fingerprint is:
SHA256:FqLWFfwoYn8Vg7MY75ugKhKg3/1qWbs1y3uVY+j047I root@2ff5eb2dc3ee
The key's randomart image is:
+---[RSA 3072]----+
|       ...       |
|      . +.o      |
|      .+o= o     |
|.   oooo+.o      |
|o  .oo.oS.   . . |
|o  .  o.+   o =  |
| o . o = +oo + . |
|o . o + +o o+ o  |
|.... ..oo.+oE+.. |
+----[SHA256]-----+
ssh-rs

In [2]:
#Adding required property to core-site.xml file
%%bash
cat <<EOF > $HADOOP_HOME/etc/hadoop/core-site.xml
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
  <property>
          <name>fs.defaultFS</name>
          <value>hdfs://localhost:9000</value>
          <description>Where HDFS NameNode can be found on the network</description>
  </property>
</configuration>
EOF

In [3]:
%%bash
cat <<EOF > $HADOOP_HOME/etc/hadoop/hdfs-site.xml
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
<property>
    <name>dfs.replication</name>
    <value>1</value>
  </property>

</configuration>
EOF

In [4]:
%%bash
cat <<EOF > $HADOOP_HOME/etc/hadoop/mapred-site.xml
<?xml version="1.0"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
<property>
    <name>mapreduce.framework.name</name>
    <value>yarn</value>
  </property>
  <property>
    <name>mapreduce.application.classpath</name>
    <value>$HADOOP_HOME/share/hadoop/mapreduce/*:$HADOOP_HOME/share/hadoop/mapreduce/lib/*</value>
  </property>

</configuration>
EOF

In [5]:
%%bash
cat <<EOF > $HADOOP_HOME/etc/hadoop/yarn-site.xml
<?xml version="1.0"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->
<configuration>
<property>
    <description>The hostname of the RM.</description>
    <name>yarn.resourcemanager.hostname</name>
    <value>localhost</value>
  </property>
  <property>
    <name>yarn.nodemanager.aux-services</name>
    <value>mapreduce_shuffle</value>
  </property>
  <property>
    <name>yarn.nodemanager.env-whitelist</name>
    <value>JAVA_HOME,HADOOP_COMMON_HOME,HADOOP_HDFS_HOME,HADOOP_CONF_DIR,CLASSPATH_PREPEND_DISTCACHE,HADOOP_YARN_HOME,HADOOP_HOME,PATH,LANG,TZ,HADOOP_MAPRED_HOME</value>
  </property>

<!-- Site specific YARN configuration properties -->

</configuration>
EOF

In [6]:
!cat $HADOOP_HOME/etc/hadoop/core-site.xml

<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
  <property>
          <name>fs.defaultFS</name>
          <value>hdfs://localhost:9000</value>
          <description>Where HDFS NameNode can be found on the network</description>
  </property>
</configuration>


# Formatting the HDFS Filesystem

In [7]:
!$HADOOP_HOME/bin/hdfs namenode -format

#Creating other necessary enviroment variables before starting nodes
os.environ["HDFS_NAMENODE_USER"] = "root"
os.environ["HDFS_DATANODE_USER"] = "root"
os.environ["HDFS_SECONDARYNAMENODE_USER"] = "root"
os.environ["YARN_RESOURCEMANAGER_USER"] = "root"
os.environ["YARN_NODEMANAGER_USER"] = "root"

#Launching hdfs deamons
!$HADOOP_HOME/sbin/start-dfs.sh

#Launching yarn deamons
#nohup causes a process to ignore a SIGHUP signal
!nohup $HADOOP_HOME/sbin/start-yarn.sh

#Listing the running deamons
!jps

namenode is running as process 1480.  Stop it first and ensure /tmp/hadoop-root-namenode.pid file is empty before retry.
Starting namenodes on [localhost]
localhost: namenode is running as process 1480.  Stop it first and ensure /tmp/hadoop-root-namenode.pid file is empty before retry.
Starting datanodes
localhost: datanode is running as process 1592.  Stop it first and ensure /tmp/hadoop-root-datanode.pid file is empty before retry.
Starting secondary namenodes [2ff5eb2dc3ee]
2ff5eb2dc3ee: secondarynamenode is running as process 1778.  Stop it first and ensure /tmp/hadoop-root-secondarynamenode.pid file is empty before retry.
nohup: ignoring input and appending output to 'nohup.out'
1778 SecondaryNameNode
4803 ResourceManager
1480 NameNode
2136 NodeManager
4972 Jps


### Monitoring Hadoop cluster with hadoop admin commands

In [8]:
#Report the basic file system information and statistics
!$HADOOP_HOME/bin/hdfs dfsadmin -report

Configured Capacity: 115658190848 (107.72 GB)
Present Capacity: 73759509080 (68.69 GB)
DFS Remaining: 73759215616 (68.69 GB)
DFS Used: 293464 (286.59 KB)
DFS Used%: 0.00%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 127.0.0.1:9866 (localhost)
Hostname: 2ff5eb2dc3ee
Decommission Status : Normal
Configured Capacity: 115658190848 (107.72 GB)
DFS Used: 293464 (286.59 KB)
Non DFS Used: 41881904552 (39.01 GB)
DFS Remaining: 73759215616 (68.69 GB)
DFS Used%: 0.00%
DFS Remaining%: 63.77%
Configured Cache Capacity: 0 (

# Hadoop Streaming Using Python

In [10]:
#Exploring Hadoop utilities available
!ls $HADOOP_HOME/share/hadoop/tools/lib/

aliyun-java-sdk-core-4.5.10.jar      hadoop-gridmix-3.2.4.jar
aliyun-java-sdk-kms-2.11.0.jar	     hadoop-kafka-3.2.4.jar
aliyun-java-sdk-ram-3.1.0.jar	     hadoop-openstack-3.2.4.jar
aliyun-sdk-oss-3.13.0.jar	     hadoop-resourceestimator-3.2.4.jar
aws-java-sdk-bundle-1.11.901.jar     hadoop-rumen-3.2.4.jar
azure-data-lake-store-sdk-2.2.9.jar  hadoop-sls-3.2.4.jar
azure-keyvault-core-1.0.0.jar	     hadoop-streaming-3.2.4.jar
azure-storage-7.0.0.jar		     ini4j-0.5.4.jar
hadoop-aliyun-3.2.4.jar		     jdom2-2.0.6.jar
hadoop-archive-logs-3.2.4.jar	     kafka-clients-2.8.1.jar
hadoop-archives-3.2.4.jar	     lz4-java-1.7.1.jar
hadoop-aws-3.2.4.jar		     ojalgo-43.0.jar
hadoop-azure-3.2.4.jar		     opentracing-api-0.33.0.jar
hadoop-azure-datalake-3.2.4.jar      opentracing-noop-0.33.0.jar
hadoop-datajoin-3.2.4.jar	     opentracing-util-0.33.0.jar
hadoop-distcp-3.2.4.jar		     org.jacoco.agent-0.8.5-runtime.jar
hadoop-extras-3.2.4.jar		     wildfly-openssl-1.0.7.Final.jar
hadoop-fs2img-3.2.4.

In [11]:
#Dowloading text example to use as input (if it has not been donwloaded yet)
!wget -q https://raw.githubusercontent.com/ayyoubmaul/hadoop-docker/refs/heads/main/data/uud45.txt

In [ ]:
#Creating directory in HDFS
!$HADOOP_HOME/bin/hdfs dfs -mkdir /word_count_with_python

In [ ]:
#Copying the file from local file system to Hadoop distributed file system (HDFS)
!$HADOOP_HOME/bin/hdfs dfs -put /content/uud45.txt /word_count_with_python

In [14]:
!$HADOOP_HOME/bin/hdfs dfs -ls /word_count_with_python

Found 2 items
drwxr-xr-x   - root supergroup          0 2025-03-13 14:47 /word_count_with_python/output
-rw-r--r--   1 root supergroup       1412 2025-03-13 14:46 /word_count_with_python/uud45.txt


In [15]:
#Testing our MapReduce job locally (Hadoop does not participate here)
!cat /content/uud45.txt | python mapper.py | sort -k1,1 | python reducer.py | head -10
#We apply sorting after the mapper because it is the default operation in MapReduce architecture

:	1
abadi	1
adil	2
Allah	1
Atas	1
bagi	1
Bahwa	1
bangsa	2
bangsa,	1
bebas,	1


In [16]:
#Changing the permissions of the files
!chmod 777 /content/mapper.py /content/reducer.py
#Setting 777 permissions to a file or directory means that it will be readable, writable and executable by all users

In [17]:
#Running MapReduce programs in hadoop
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-3.2.4.jar \
  -input /word_count_with_python/uud45.txt \
  -output /word_count_with_python/output \
  -mapper "python /content/mapper.py" \
  -reducer "python /content/reducer.py"

packageJobJar: [/tmp/hadoop-unjar196547411499004296/] [] /tmp/streamjob8771466128360379435.jar tmpDir=null
2025-03-13 14:51:07,624 INFO client.RMProxy: Connecting to ResourceManager at localhost/127.0.0.1:8032
2025-03-13 14:51:07,928 INFO client.RMProxy: Connecting to ResourceManager at localhost/127.0.0.1:8032
2025-03-13 14:51:08,112 ERROR streaming.StreamJob: Error Launching job : Output directory hdfs://localhost:9000/word_count_with_python/output already exists
Streaming Command Failed!


In [18]:
#Exploring the created output directory
#part-r-00000 contains the actual ouput
!$HADOOP_HOME/bin/hdfs dfs -ls /word_count_with_python/output

Found 2 items
-rw-r--r--   1 root supergroup          0 2025-03-13 14:47 /word_count_with_python/output/_SUCCESS
-rw-r--r--   1 root supergroup       1276 2025-03-13 14:47 /word_count_with_python/output/part-00000


In [19]:
#Printing out first 50 lines
!$HADOOP_HOME/bin/hdfs dfs -cat /word_count_with_python/output/part-00000 | head -50

2025-03-13 14:51:17,699 WARN impl.BlockReaderFactory: I/O error constructing remote block reader.
java.net.ConnectException: Connection refused
	at sun.nio.ch.SocketChannelImpl.checkConnect(Native Method)
	at sun.nio.ch.SocketChannelImpl.finishConnect(SocketChannelImpl.java:716)
	at org.apache.hadoop.net.SocketIOWithTimeout.connect(SocketIOWithTimeout.java:205)
	at org.apache.hadoop.net.NetUtils.connect(NetUtils.java:535)
	at org.apache.hadoop.hdfs.DFSClient.newConnectedPeer(DFSClient.java:2973)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.nextTcpPeer(BlockReaderFactory.java:826)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.getRemoteBlockReaderFromTcp(BlockReaderFactory.java:751)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.build(BlockReaderFactory.java:381)
	at org.apache.hadoop.hdfs.DFSInputStream.getBlockReader(DFSInputStream.java:733)
	at org.apache.hadoop.hdfs.DFSInputStream.blockSeekTo(DFSInputStream.java:664)
	at org.apache.hadoop.hdfs.DFSI

In [ ]:
# copy file hdfs-wordcount.txt dari HDFS ke local
!$HADOOP_HOME/bin/hdfs dfs -copyToLocal /word_count_with_python/output/part-00000 /content/output-hdfs-wordcount.txt